In [ ]:
%%capture
import os
import pandas as pd
import numpy as np
from dj_notebook import activate
from pathlib import Path

env_file = os.environ["INTECOMM_ENV"]
analysis_folder = Path(os.environ["INTECOMM_ANALYSIS_FOLDER"])
reports_folder = Path(os.environ["INTECOMM_ANALYSIS_FOLDER"])
plus = activate(dotenv_file=env_file)

In [ ]:
from intecomm_analytics.dataframes import get_df_main_1858
from intecomm_analytics.notebooks.primary.table_utils import get_formatted_rows_by_country

In [ ]:
df_main = get_df_main_1858(None)

In [ ]:
cutoff_days = 0

df_main["hiv_dx_days"] = df_main[(df_main.hiv_only==1) & (df_main.hiv_timedelta_dx.dt.days > cutoff_days)].hiv_timedelta_dx.dt.days
df_main["htn_dx_days"] = df_main[(df_main.htn_only==1) & (df_main.htn_timedelta_dx.dt.days > cutoff_days)].htn_timedelta_dx.dt.days
df_main["dm_dx_days"] = df_main[(df_main.dm_only==1) & (df_main.dm_timedelta_dx.dt.days > cutoff_days)].dm_timedelta_dx.dt.days

def get_max_days(s):
    dm = s.dm_timedelta_dx.days
    htn = s.htn_timedelta_dx.days
    return max(dm, htn)

def get_min_days(s):
    dm = s.dm_timedelta_dx.days
    htn = s.htn_timedelta_dx.days
    return min(dm, htn)

df_main["ncd_dx_max_days"] = df_main[(df_main.ncd==1) & ((df_main.dm_timedelta_dx.dt.days > cutoff_days) | (df_main.htn_timedelta_dx.dt.days > cutoff_days))].apply(get_max_days, axis=1)

df_main["ncd_dx_min_days"] = df_main[(df_main.ncd==1) & ((df_main.dm_timedelta_dx.dt.days > cutoff_days) | (df_main.htn_timedelta_dx.dt.days > cutoff_days))].apply(get_min_days, axis=1)



In [ ]:
table_dfs = []

table = {'Condition': ['HIV', '', '', '', '', '']}
table.update({
    'Parameter': ['Days since diagnosis', '', '', '', '', ''],
    **get_formatted_rows_by_country(df_main[df_main.hiv==1], "hiv_dx_days", "hiv_dx_days")
})
table_df = pd.DataFrame(table)
table_df.drop(columns=["Timepoint"], inplace=True)
table_df = table_df.drop(table_df.index[[3, 4, 5]])
table_dfs.append(table_df)

table = {'Condition': ['HTN', '', '', '', '', '']}
table.update({
    'Parameter': ['Days since diagnosis', '', '', '', '', ''],
    **get_formatted_rows_by_country(df_main[df_main.htn==1], "htn_dx_days", "htn_dx_days")
})
table_df = pd.DataFrame(table)
table_df.drop(columns=["Timepoint"], inplace=True)
table_df = table_df.drop(table_df.index[[3, 4, 5]])
table_dfs.append(table_df)

table = {'Condition': ['DM', '', '', '', '', '']}
table.update({
    'Parameter': ['Days since diagnosis', '', '', '', '', ''],
    **get_formatted_rows_by_country(df_main[df_main.dm==1], "dm_dx_days", "dm_dx_days")
})
table_df = pd.DataFrame(table)
table_df.drop(columns=["Timepoint"], inplace=True)
table_df = table_df.drop(table_df.index[[3, 4, 5]])
table_dfs.append(table_df)


table = {'Condition': ['HIV only', '', '', '', '', '']}
table.update({
    'Parameter': ['Days since diagnosis', '', '', '', '', ''],
    **get_formatted_rows_by_country(df_main[df_main.hiv_only==1], "hiv_dx_days", "hiv_dx_days")
})
table_df = pd.DataFrame(table)
table_df.drop(columns=["Timepoint"], inplace=True)
table_df = table_df.drop(table_df.index[[3, 4, 5]])
table_dfs.append(table_df)

table = {'Condition': ['HTN only', '', '', '', '', '']}
table.update({
    'Parameter': ['Days since diagnosis', '', '', '', '', ''],
    **get_formatted_rows_by_country(df_main[df_main.htn_only==1], "htn_dx_days", "htn_dx_days")
})
table_df = pd.DataFrame(table)
table_df.drop(columns=["Timepoint"], inplace=True)
table_df = table_df.drop(table_df.index[[3, 4, 5]])
table_dfs.append(table_df)

table = {'Condition': ['DM only', '', '', '', '', '']}
table.update({
    'Parameter': ['Days since diagnosis', '', '', '', '', ''],
    **get_formatted_rows_by_country(df_main[df_main.dm_only==1], "dm_dx_days", "dm_dx_days")
})
table_df = pd.DataFrame(table)
table_df.drop(columns=["Timepoint"], inplace=True)
table_df = table_df.drop(table_df.index[[3, 4, 5]])
table_dfs.append(table_df)

table = {'Condition': ['NCD', '', '', '', '', '']}
table.update({
    'Parameter': ['Days since diagnosis (min)', '', '', '', '', ''],
    **get_formatted_rows_by_country(df_main[df_main.ncd==1], "ncd_dx_min_days", "ncd_dx_min_days")
})
table_df = pd.DataFrame(table)
table_df.drop(columns=["Timepoint"], inplace=True)
table_df = table_df.drop(table_df.index[[3, 4, 5]])
table_dfs.append(table_df)

table = {'Condition': ['NCD', '', '', '', '', '']}
table.update({
    'Parameter': ['Days since diagnosis (max)', '', '', '', '', ''],
    **get_formatted_rows_by_country(df_main[df_main.ncd==1], "ncd_dx_max_days", "ncd_dx_max_days")
})
table_df = pd.DataFrame(table)
table_df.drop(columns=["Timepoint"], inplace=True)
table_df = table_df.drop(table_df.index[[3, 4, 5]])
table_dfs.append(table_df)


In [ ]:
table_df = pd.concat(table_dfs, ignore_index=True)


In [ ]:
table_df

In [ ]:
path = analysis_folder / 'dx_days_by_country.csv'
table_df.to_csv(path, index=False)